#### This notebook add centroid for origin and destination zone of trips
* The centroids of zones are generated from SA1 shapefile of ABS census
* 


In [422]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import config

In [423]:
trips = pd.read_csv(config.TRIPS_1216_PRC, low_memory=False)
trips.shape

(44319, 20)

In [424]:
trips.head()

,TRIPID,PERSID,TRIPNO,TRAVDOW,LINKMODE,TRIPPURP,DESTPURP1,STARTHOUR,STARTIME,ARRHOUR,ARRTIME,TRIPTIME,TRAVTIME,WAITIME,ORIGSA1,ORIGSA3,DESTSA1,DESTSA3,CUMDIST,Mode
0,Y12H0000107P01T01,Y12H0000107P01,1,Sunday,Vehicle Driver,Social,Social,10,630,10,650,20,20,0,20904122028,Whittlesea - Wallan,20904122002,Whittlesea - Wallan,2.60,Car
1,Y12H0000107P01T02,Y12H0000107P01,2,Sunday,Vehicle Driver,Social,At or Go Home,12,750,12,760,10,10,0,20904122002,Whittlesea - Wallan,20904122028,Whittlesea - Wallan,2.60,Car
2,Y12H0000107P01T03,Y12H0000107P01,3,Sunday,Walking,Recreational,Recreational,13,810,14,840,30,30,0,20904122028,Whittlesea - Wallan,20904122031,Whittlesea - Wallan,3.23,Walk
3,Y12H0000107P01T04,Y12H0000107P01,4,Sunday,Walking,Recreational,At or Go Home,14,855,14,885,30,30,0,20904122031,Whittlesea - Wallan,20904122028,Whittlesea - Wallan,3.23,Walk
4,Y12H0000107P02T01,Y12H0000107P02,1,Sunday,Vehicle Passenger,Social,Social,10,630,10,650,20,20,0,20904122028,Whittlesea - Wallan,20904122002,Whittlesea - Wallan,2.60,Car


In [425]:
trips.columns

Index(['TRIPID', 'PERSID', 'TRIPNO', 'TRAVDOW', 'LINKMODE', 'TRIPPURP',
       'DESTPURP1', 'STARTHOUR', 'STARTIME', 'ARRHOUR', 'ARRTIME', 'TRIPTIME',
       'TRAVTIME', 'WAITIME', 'ORIGSA1', 'ORIGSA3', 'DESTSA1', 'DESTSA3',
       'CUMDIST', 'Mode'],
      dtype='object')

### Get SA1 shapefile

In [426]:
shape_file = "../data/raw/1270055001_sa1_2016_aust_shape.zip"
gdf = gpd.read_file(shape_file)

In [427]:
gdf.shape

(57523, 15)

In [428]:
gdf[gdf['SA1_MAIN16'].isin([20604112201])]

,SA1_MAIN16,SA1_7DIG16,SA2_MAIN16,SA2_5DIG16,SA2_NAME16,SA3_CODE16,SA3_NAME16,SA4_CODE16,SA4_NAME16,GCC_CODE16,GCC_NAME16,STE_CODE16,STE_NAME16,AREASQKM16,geometry


In [429]:
gdf.head()

,SA1_MAIN16,SA1_7DIG16,SA2_MAIN16,SA2_5DIG16,SA2_NAME16,SA3_CODE16,SA3_NAME16,SA4_CODE16,SA4_NAME16,GCC_CODE16,GCC_NAME16,STE_CODE16,STE_NAME16,AREASQKM16,geometry
0,10102100701,1100701,101021007,11007,Braidwood,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,362.8727,"POLYGON ((149.71174 -35.12318, 149.71184 -35.1..."
1,10102100702,1100702,101021007,11007,Braidwood,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,229.7459,"POLYGON ((149.73421 -35.36758, 149.74475 -35.3..."
2,10102100703,1100703,101021007,11007,Braidwood,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,2.3910,"POLYGON ((149.77998 -35.44067, 149.77996 -35.4..."
3,10102100704,1100704,101021007,11007,Braidwood,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,1.2816,"POLYGON ((149.79744 -35.44149, 149.79778 -35.4..."
4,10102100705,1100705,101021007,11007,Braidwood,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,1.1978,"POLYGON ((149.79498 -35.44902, 149.79517 -35.4..."


In [430]:
gdf.STE_NAME16.value_counts()

New South Wales                 18399
Victoria                        14073
Queensland                      11563
Western Australia                5984
South Australia                  4245
Tasmania                         1464
Australian Capital Territory     1147
Northern Territory                626
Other Territories                  22
Name: STE_NAME16, dtype: int64

In [431]:
# Only using SA1 from Victoria
# gdf_vic = gdf[gdf.STE_NAME16 == "Victoria"]
gdf_vic = gdf
gdf_vic.shape

(57523, 15)

In [432]:
gdf_vic.columns

Index(['SA1_MAIN16', 'SA1_7DIG16', 'SA2_MAIN16', 'SA2_5DIG16', 'SA2_NAME16',
       'SA3_CODE16', 'SA3_NAME16', 'SA4_CODE16', 'SA4_NAME16', 'GCC_CODE16',
       'GCC_NAME16', 'STE_CODE16', 'STE_NAME16', 'AREASQKM16', 'geometry'],
      dtype='object')

In [433]:
# Remove SA1 do not have geometry
gdf_vic.dropna(inplace=True)

In [434]:
gdf_vic.isna().sum()

SA1_MAIN16    0
SA1_7DIG16    0
SA2_MAIN16    0
SA2_5DIG16    0
SA2_NAME16    0
SA3_CODE16    0
SA3_NAME16    0
SA4_CODE16    0
SA4_NAME16    0
GCC_CODE16    0
GCC_NAME16    0
STE_CODE16    0
STE_NAME16    0
AREASQKM16    0
geometry      0
dtype: int64

In [435]:
columns = ['SA1_MAIN16', 'SA2_MAIN16', 'SA2_NAME16', 
           'SA3_CODE16', 'SA3_NAME16', 'SA4_CODE16', 'SA4_NAME16', 
           'STE_NAME16', 'AREASQKM16', 'geometry']
gdf_vic = gdf_vic[columns]
gdf_vic.shape

(57490, 10)

In [436]:
gdf_vic["centroid"] = gdf_vic.geometry.apply(lambda x: x.centroid)

/home/danph/.pyenv/versions/3.8.5/envs/.ml/lib/python3.8/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


In [437]:
gdf_vic.head()

,SA1_MAIN16,SA2_MAIN16,SA2_NAME16,SA3_CODE16,SA3_NAME16,SA4_CODE16,SA4_NAME16,STE_NAME16,AREASQKM16,geometry,centroid
0,10102100701,101021007,Braidwood,10102,Queanbeyan,101,Capital Region,New South Wales,362.8727,"POLYGON ((149.71174 -35.12318, 149.71184 -35.1...",POINT (149.85770 -35.14610)
1,10102100702,101021007,Braidwood,10102,Queanbeyan,101,Capital Region,New South Wales,229.7459,"POLYGON ((149.73421 -35.36758, 149.74475 -35.3...",POINT (149.80306 -35.43468)
2,10102100703,101021007,Braidwood,10102,Queanbeyan,101,Capital Region,New South Wales,2.3910,"POLYGON ((149.77998 -35.44067, 149.77996 -35.4...",POINT (149.78959 -35.43862)
3,10102100704,101021007,Braidwood,10102,Queanbeyan,101,Capital Region,New South Wales,1.2816,"POLYGON ((149.79744 -35.44149, 149.79778 -35.4...",POINT (149.80227 -35.43901)
4,10102100705,101021007,Braidwood,10102,Queanbeyan,101,Capital Region,New South Wales,1.1978,"POLYGON ((149.79498 -35.44902, 149.79517 -35.4...",POINT (149.79680 -35.44997)


### Merge OD SA1 centroids to trips dataset

In [438]:
trips.head()

,TRIPID,PERSID,TRIPNO,TRAVDOW,LINKMODE,TRIPPURP,DESTPURP1,STARTHOUR,STARTIME,ARRHOUR,ARRTIME,TRIPTIME,TRAVTIME,WAITIME,ORIGSA1,ORIGSA3,DESTSA1,DESTSA3,CUMDIST,Mode
0,Y12H0000107P01T01,Y12H0000107P01,1,Sunday,Vehicle Driver,Social,Social,10,630,10,650,20,20,0,20904122028,Whittlesea - Wallan,20904122002,Whittlesea - Wallan,2.60,Car
1,Y12H0000107P01T02,Y12H0000107P01,2,Sunday,Vehicle Driver,Social,At or Go Home,12,750,12,760,10,10,0,20904122002,Whittlesea - Wallan,20904122028,Whittlesea - Wallan,2.60,Car
2,Y12H0000107P01T03,Y12H0000107P01,3,Sunday,Walking,Recreational,Recreational,13,810,14,840,30,30,0,20904122028,Whittlesea - Wallan,20904122031,Whittlesea - Wallan,3.23,Walk
3,Y12H0000107P01T04,Y12H0000107P01,4,Sunday,Walking,Recreational,At or Go Home,14,855,14,885,30,30,0,20904122031,Whittlesea - Wallan,20904122028,Whittlesea - Wallan,3.23,Walk
4,Y12H0000107P02T01,Y12H0000107P02,1,Sunday,Vehicle Passenger,Social,Social,10,630,10,650,20,20,0,20904122028,Whittlesea - Wallan,20904122002,Whittlesea - Wallan,2.60,Car


In [439]:
trips = trips.merge(gdf_vic[['SA1_MAIN16','centroid']], how="left", left_on="ORIGSA1", right_on="SA1_MAIN16")
trips = trips.rename(columns={"centroid":"ORIG_CENTROID"})
trips = trips.drop(columns=["SA1_MAIN16"], axis=1)

In [440]:
trips = trips.merge(gdf_vic[['SA1_MAIN16','centroid']], how="left", left_on="DESTSA1", right_on="SA1_MAIN16")
trips = trips.rename(columns={"centroid":"DEST_CENTROID"})
trips = trips.drop(columns=["SA1_MAIN16"], axis=1)

In [441]:
trips.head()

,TRIPID,PERSID,TRIPNO,TRAVDOW,LINKMODE,TRIPPURP,DESTPURP1,STARTHOUR,STARTIME,ARRHOUR,...,TRAVTIME,WAITIME,ORIGSA1,ORIGSA3,DESTSA1,DESTSA3,CUMDIST,Mode,ORIG_CENTROID,DEST_CENTROID
0,Y12H0000107P01T01,Y12H0000107P01,1,Sunday,Vehicle Driver,Social,Social,10,630,10,...,20,0,20904122028,Whittlesea - Wallan,20904122002,Whittlesea - Wallan,2.60,Car,POINT (145.08730 -37.65974),POINT (145.07152 -37.65148)
1,Y12H0000107P01T02,Y12H0000107P01,2,Sunday,Vehicle Driver,Social,At or Go Home,12,750,12,...,10,0,20904122002,Whittlesea - Wallan,20904122028,Whittlesea - Wallan,2.60,Car,POINT (145.07152 -37.65148),POINT (145.08730 -37.65974)
2,Y12H0000107P01T03,Y12H0000107P01,3,Sunday,Walking,Recreational,Recreational,13,810,14,...,30,0,20904122028,Whittlesea - Wallan,20904122031,Whittlesea - Wallan,3.23,Walk,POINT (145.08730 -37.65974),POINT (145.06223 -37.66497)
3,Y12H0000107P01T04,Y12H0000107P01,4,Sunday,Walking,Recreational,At or Go Home,14,855,14,...,30,0,20904122031,Whittlesea - Wallan,20904122028,Whittlesea - Wallan,3.23,Walk,POINT (145.06223 -37.66497),POINT (145.08730 -37.65974)
4,Y12H0000107P02T01,Y12H0000107P02,1,Sunday,Vehicle Passenger,Social,Social,10,630,10,...,20,0,20904122028,Whittlesea - Wallan,20904122002,Whittlesea - Wallan,2.60,Car,POINT (145.08730 -37.65974),POINT (145.07152 -37.65148)


In [442]:
trips.isna().sum()

TRIPID              0
PERSID              0
TRIPNO              0
TRAVDOW             0
LINKMODE            0
TRIPPURP            0
DESTPURP1           0
STARTHOUR           0
STARTIME            0
ARRHOUR             0
ARRTIME             0
TRIPTIME            0
TRAVTIME            0
WAITIME             0
ORIGSA1             0
ORIGSA3             0
DESTSA1             0
DESTSA3             0
CUMDIST             0
Mode                0
ORIG_CENTROID    5246
DEST_CENTROID    5237
dtype: int64

In [443]:
trips[trips.isna()].shape

(44319, 22)

In [444]:
trips[trips.isna()].head()

,TRIPID,PERSID,TRIPNO,TRAVDOW,LINKMODE,TRIPPURP,DESTPURP1,STARTHOUR,STARTIME,ARRHOUR,...,TRAVTIME,WAITIME,ORIGSA1,ORIGSA3,DESTSA1,DESTSA3,CUMDIST,Mode,ORIG_CENTROID,DEST_CENTROID
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None


In [445]:
trips = trips.dropna()

#### Get trip ORIG_XY and DEST_XY

In [446]:
trips.loc[2,].ORIG_CENTROID.xy

(array('d', [145.08730214277455]), array('d', [-37.65974093958047]))

In [447]:
def get_x(trip, column):
    return trip[column].x

def get_y(trip, column):
    return trip[column].y

In [448]:
get_xy(trips.loc[0,], 'ORIG_CENTROID')

(145.08730214277455, -37.65974093958047)

In [449]:
trips['ORIG_X'] = trips.apply(lambda row: get_x(row, 'ORIG_CENTROID'), axis=1)
trips['ORIG_Y'] = trips.apply(lambda row: get_y(row, 'ORIG_CENTROID'), axis=1)
trips['DEST_X'] = trips.apply(lambda row: get_x(row, 'DEST_CENTROID'), axis=1)
trips['DEST_Y'] = trips.apply(lambda row: get_y(row, 'DEST_CENTROID'), axis=1)

In [450]:
trips.TRAVDOW.value_counts()

Thursday     5977
Wednesday    5665
Monday       5609
Friday       5571
Tuesday      5316
Saturday     4297
Sunday       3852
Name: TRAVDOW, dtype: int64

### Saves trips OD locations into CSV

In [451]:
trips.to_csv("../data/processed/trip_od_locations.csv", index=False)

In [452]:
trips.shape

(36287, 26)

### Get person and household main info

##### Get household info

In [453]:
hh = pd.read_csv("../data/raw/VISTA_2012_16_v1_SA1_CSV/H_VISTA12_16_SA1_V1.csv")
hh.shape

/home/danph/.pyenv/versions/3.8.5/envs/.ml/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (32) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


(18152, 53)

In [454]:
cols = ['HHID', 'HHSIZE', 'VISITORS', 'DWELLTYPE',
       'OWNDWELL', 'YEARSLIVED', 'MONTHSLIVED', 'ADULTBIKES', 'KIDSBIKES',
       'TOTALBIKES', 'PHONE', 'CARS', 'FOURWDS', 'UTES', 'VANS', 'TRUCKS',
       'MBIKES', 'OTHERVEHS', 'TOTALVEHS', 'TRAVDATE', 'TRAVMONTH', 'TRAVYEAR',
       'HOMETAZ', 'YOUNGEST', 'AVEAGE', 'OLDEST', 'HHINC']

In [455]:
hh = hh[cols]
hh.shape

(18152, 27)

##### Get person info

In [456]:
ps = pd.read_csv("../data/raw/VISTA_2012_16_v1_SA1_CSV/P_VISTA12_16_SA1_V1.csv")
ps.shape

(46562, 78)

In [457]:
cols =['PERSID', 'HHID', 'PERSNO', 'WEEK', 'MONTHOFBIRTH',
       'YEAROFBIRTH', 'AGE', 'AGEGROUP', 'SEX', 'RELATIONSHIP', 'CARLICENCE',
       'MBIKELICENCE', 'OTHERLICENCE', 'NOLICENCE', 'FULLTIMEWORK',
       'PARTTIMEWORK', 'CASUALWORK', 'ANYWORK', 'STUDYING', 'ACTIVITIES',
       'MAINACT', 'WORKTYPE', 'EMPTYPE', 'ANZSCO1', 'ANZSIC1',
       'STARTPLACE', 'ANYSTOPS', 'NUMSTOPS', 'REASONCODE', 'MRTDOW',
       'MRTINT', 'PERSINC', 'WHOFILLED', 'PROXY', 'FILLDOW', 'FILLLAG',
       'ADDITIONALTRAVEL', 'NEWSTOP', 'DISABLEDTRANS', 'TAXITRANS',
       'WHEELCHAIRTRANS', 'ACCESSTRANS', 'NONETRANS', 'NOTTOSAYTRANS',
       'CYCLEDWORK', 'CYCLEDSHOPPING', 'CYCLEDEXERCISE', 'CYCLEDOTHER',
       'NOCYCLED', 'HOMESLA', 'HOMELGA', 'HomeSA1', 'HomeSA2', 'HomeSA3',
       'HomeSA4', 'HomeRegion_ASGS', 'HomeSubRegion_ASGS', 'AgeGroup_RW', 'STOPS',
       'ReportingPeriod']

In [458]:
ps = ps[cols]
ps.shape

(46562, 60)

##### Merge household info into person table

In [459]:
ps = ps.merge(hh, how='left', on='HHID')
ps.shape

(46562, 86)

In [460]:
ps.to_csv("../data/processed/persons.csv", index=False)

### Merge person info into Trip table

In [461]:
trips = trips.merge(ps, how="left", on='PERSID')
trips.shape

(36287, 111)

In [462]:
trips.to_csv("../data/processed/trip_od_full.csv", index=False)